## Introduction

The Makridakis competitions (or M-competitions), organised by forecasting expert Spyros Makridakis, aim to provide a better understanding and advancement of forecasting methodology by comparing the performance of different methods in solving a well-defined, real-world problem. The first M-competition was held in 1982. According to forecasting researcher and practitioner Rob Hyndman the M-competitions “have had an enormous influence on the field of forecasting. They focused attention on what models produced good forecasts, rather than on the mathematical properties of those models”. This empirical approach is very similar to Kaggle’s trade-mark way of having the best machine learning algorithms engage in intense competition on diverse datasets. M5 is the first M-competition to be held on Kaggle.


## Objective

To predict sales data provided by the retail giant Walmart 4 weeks into the future (two 2-week windows). 


## Dataset

The data: We are working with 42,840 hierarchical time series. The data were obtained in the 3 US states of California (CA), Texas (TX), and Wisconsin (WI). “Hierarchical” here means that data can be aggregated on different levels: item level, department level, product category level, and state level. The sales information reaches back from Jan 2011 to June 2016. In addition to the sales numbers, we are also given corresponding data on prices, promotions, and holidays. The dataset is zero-inflated, that is entries with empty information are imputed with a zero value. Hence, most of the time series contain zero values.

The data comprises 3049 individual products from 3 categories and 7 departments, sold in 10 stores in 3 states. The hierachical aggregation captures the combinations of these factors. For instance, we can create 1 time series for all sales, 3 time series for all sales per state, and so on. The largest category is sales of all individual 3049 products per 10 stores for 30490 time series.


## Dataset files
The training data comes in the shape of 3 separate files:

`sales_train.csv`: this is our main training data. It has 1 column for each of the 1941 days from 2011-01-29 and 2016-05-22; not including the validation period of 28 days until 2016-06-19. It also includes the IDs for item, department, category, store, and state. The number of rows is 30490 for all combinations of 30490 items and 10 stores.

`sell_prices.csv`: the store and item IDs together with the sales price of the item as a weekly average.

`calendar.csv`: dates together with related features like day-of-the week, month, year, and an 3 binary flags for whether the stores in each state allowed purchases with SNAP food stamps at this date (1) or not (0).


## Metrics

The point forecast submission are being evaluated using the **Root Mean Squared Scaled Error (RMSSE)**, which is derived from the Mean Absolute Scaled Error (MASE) that was designed to be scale invariant and symmetric. In a similar way to the MASE, the RMSSE is scale invariant and symmetric, and measures the prediction error (i.e. forecast - truth) relative to a “naive forecast” that simply assumes that step i = step i-1. In contrast to the MASE, here both prediction error and naive error are scaled to account for the goal of estimating average values in the presence of many zeros.

The metric is computed for each time series and then averaged accross all time series including weights. The weights are proportional to the sales volume of the item, in dollars, to give more importance to high selling products. Note, that the weights are based on the last 28 days of the training data, and that those dates will be adjusted for the ultimate evaluation data, as confirmed by the organisers.

### Table of Contents

* [ Load data and downcasting ](#downcast)

* [ Preprocessing ](#preprocess)

* [ CNN-LSTM Model](#cnnlstm)

<a name="downcast"></a>
## Load data and downcasting

Downcasting the dataframes helps to reduce the amount of storage used by them and also to expidite the operations performed on them.

Numerical Columns: Depending on your environment, pandas automatically creates int32, int64, float32 or float64 columns for numeric ones. If you know the min or max value of a column, you can use a subtype which is less memory consuming. You can also use an unsigned subtype if there is no negative value. In this dataset, we convert features with a float64 data type into float32 data type, and convert features with int64, int32 to in16 data type.

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

train_sales = pd.read_csv('Data/sales_train_evaluation.csv')
calendar = pd.read_csv('Data/calendar.csv')

In [4]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

#Reduce memory usage and compare with the previous one to be sure
train_sales = downcast_dtypes(train_sales)

In [5]:
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


<a name="preprocess"></a>
## Preprocess dataset

Based on the sell price dataset, there are multiple products which has zero prices from day 1 to day 350. This is due to the product not being listed in the store yet. Additionally, on the sales_train dataset, there are multiple products which has zero sales, This could be due to zero prices or that the sales is not recorded. 

Since our LSTM model predicts the future sales of multiple products instead of the sales of individual products, hence we need to set a cutoff data instead of trimming the dataset by the first non-zero prices. Hence we set this cutoff date on day 350 

In [6]:
# Preprocess: remove id, item_id, dept_id, cat_id, store_id, state_id columns
startDay = 350  # Remove the first 350 days in train sales data due to zero_inflated data
train_sales = train_sales.T
train_sales = train_sales[6 + startDay:]
train_sales.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


### Additional features: event 1, event 2 and SNAP

Besides using sales as a time series feature, festive or sports events can have a strong positive influence on the sales. For example, shoppers are more likely to purchase more snacks or food a day before Thanksgiving or Superbowl. Similarly, we expect more sales during the days with SNAP program. Hence, we inserted 5 additional features: event 1, event 2, SNAP_CA, SNAP_WI, SNAP_TX. The days of SNAP program is different for each store location and therefore we need 3 separate features for SNAP.  

In [7]:
# Initialize a dataframe with zeros for 1969 days in the calendar

daysBeforeEvent1 = pd.DataFrame(np.zeros((1969,1)))
daysBeforeEvent2 = pd.DataFrame(np.zeros((1969,1)))

snap_CA = pd.DataFrame(np.zeros((1969,1)))
snap_TX = pd.DataFrame(np.zeros((1969,1)))
snap_WI = pd.DataFrame(np.zeros((1969,1)))


# Label 1 to one day before the event_name_1 
# Label 1 to one day before the event_name_2
# Sales are likely to increase one day before events like superbowl etc.

# Label 1 to days on snap_CA
# Label 1 to days on snap_TX
# Label 1 to days on snap_WI

for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent1[0][x-1] = 1 
            
    if((pd.isnull(calendar["event_name_2"][x])) == False):
           daysBeforeEvent2[0][x-1] = 1    
    
    
    if((pd.isnull(calendar["snap_CA"][x])) == False):
           snap_CA[0][x] = 1    
        
    if((pd.isnull(calendar["snap_TX"][x])) == False):
           snap_TX[0][x] = 1    
        
    if((pd.isnull(calendar["snap_WI"][x])) == False):
           snap_WI[0][x] = 1

In [8]:
daysBeforeEvent1

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1964,0.0
1965,0.0
1966,0.0
1967,1.0


In [9]:
daysBeforeEvent2

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1964,0.0
1965,0.0
1966,0.0
1967,1.0


### Creating training, validation, evaluation dataset

**Training Dataset**
- To train a timeseries dataset, we will use the training dataset, which comprises of features (sales, event 1, event 2, SNAP) from day 350 to 1912. 

- From this training dataset, we will predict the future sales of each product for 56 days in 2 week windows (predict day 1913 to 1940, and predict day 1941 to 1969).

**Validation Dataset**
- The dataset with the features (sales, event 1, event 2, SNAP) from day 1913 to 1940 is used to validate the predicted values from trained dataset. 

**Evaluation Dataset**
- The dataset with the features (sales, event 1, event 2, SNAP) from day 1941 to 1969 is used to evaluate the predicted values from trained dataset. 


In [10]:
# split dataset into evaluation (last 2 weeks), validation (first 2 weeks), training  
# input for predicting validation period day 1941 to 1969

daysBeforeEvent1_eval = daysBeforeEvent1[1941:]
daysBeforeEvent2_eval = daysBeforeEvent2[1941:]

snap_CA_eval = snap_CA[1941:]
snap_TX_eval = snap_TX[1941:]
snap_WI_eval = snap_WI[1941:]


# input for predicting validation period day 1913 to 1941

daysBeforeEvent1_valid = daysBeforeEvent1[1913:1941] 
daysBeforeEvent2_valid = daysBeforeEvent2[1913:1941]

snap_CA_valid = snap_CA[1913:1941] 
snap_TX_valid = snap_TX[1913:1941]
snap_WI_valid = snap_WI[1913:1941]

# input for training as a feature
# daysBeforeEvent1 = daysBeforeEvent1[startDay:1913] 
# daysBeforeEvent2 = daysBeforeEvent2[startDay:1913] 
daysBeforeEvent1 = daysBeforeEvent1[startDay:1941] 
daysBeforeEvent2 = daysBeforeEvent2[startDay:1941]

snap_CA = snap_CA[startDay:1941] 
snap_TX = snap_TX[startDay:1941] 
snap_WI = snap_WI[startDay:1941] 

In [11]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent1.columns = ["oneDayBeforeEvent1"]
daysBeforeEvent1.index = train_sales.index

daysBeforeEvent2.columns = ["oneDayBeforeEvent2"]
daysBeforeEvent2.index = train_sales.index


snap_CA.columns = ["snap_CA"]
snap_CA.index = train_sales.index

snap_TX.columns = ["snap_TX"]
snap_TX.index = train_sales.index

snap_WI.columns = ["snap_WI"]
snap_WI.index = train_sales.index

In [12]:
train_sales = pd.concat([train_sales, daysBeforeEvent1, daysBeforeEvent2,
                        snap_CA, snap_TX, snap_WI], axis = 1, sort=False)
train_sales.head()  # additional features (event1, event2, SNAP) are added

,0,1,2,3,4,5,6,7,8,9,...,30485,30486,30487,30488,30489,oneDayBeforeEvent1,oneDayBeforeEvent2,snap_CA,snap_TX,snap_WI
d_351,0,0,0,2,0,0,0,24,3,2,...,0,0,1,0,0,0.0,0.0,1.0,1.0,1.0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,1,2,0,0,1.0,0.0,1.0,1.0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,1,2,0,0,0.0,0.0,1.0,1.0,1.0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,0,0,0,0,0.0,0.0,1.0,1.0,1.0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,1,1,0,0,0.0,0.0,1.0,1.0,1.0


### Standardizing features

- It is also important to scale our features across the columns. Each columns represents the sales values of a particular day. This helps to ensure that the sales values are between 0 and 1, and this helps the gradient descent optimization algorithm in LSTM model.

In [13]:
#Feature Scaling: Scale features using min-max scaler in range 0-1
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
train_sales.columns = train_sales.columns.astype(str)
train_sales_scaled = sc.fit_transform(train_sales)

In [14]:
timesteps = 28  # use the last 28 days to predict the next day's sales
X_train = []
y_train = []

for i in range(timesteps, 1941 - startDay):
    X_train.append(train_sales_scaled[i-timesteps:i])
    y_train.append(train_sales_scaled[i][0:30490])

In [15]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1563, 28, 30495)
(1563, 30490)


In [16]:
X_train.shape

(1563, 28, 30495)

In [17]:
y_train.shape

(1563, 30490)

<a name="cnnlstm"></a>
## CNN-LSTM Model

The model used is CNN-LSTM, with regularization parameters such as batch normalization. There are 2 units of CNN (conv1d followed by max pooling layer). The conv1d layer helps to analyse and extract the patterns of the sales in 1 week window (just like extracting "edges" patterns in a typcial conv2d on an image). The max pooling layer then summarizes these patterns into broader, generalizable patterns, which  seeks to remove noises in the spikes or troughs in daily sales.

The outputs from CNN is then fed into a LSTM model with 3 layers of bidirectional LSTM. We use an inverted number of LSTM units from 512 to 256 to 128. This has the property of reducing and summarizing the sales patterns for better prediction. The batch normalization helps to improve the robustness of the model by reducing the likelihood of overfitting. 

In [18]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
  except RuntimeError as e:
    print(e)

In [19]:
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
  except RuntimeError as e:
    print(e)

In [20]:
print(tf.__version__)

2.14.0


In [21]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout


tf.random.set_seed(51)
np.random.seed(51)

n_timesteps = X_train.shape[1]
n_products = X_train.shape[2]


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=7,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(n_timesteps, n_products)),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=64, kernel_size=7, 
                           strides=1, activation='relu', padding="causal"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30490)
])


opt_adam = tf.keras.optimizers.Adam(clipvalue=0.5)

model.compile(loss='mse',
              optimizer=opt_adam, 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 128)           27323648  
                                                                 
 max_pooling1d (MaxPooling1  (None, 14, 128)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 64)            57408     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 7, 64)             0         
 g1D)                                                            
                                                                 
 bidirectional (Bidirection  (None, 7, 1024)           2363392   
 al)                                                             
                                                        

## Callbacks

Callbacks are inserted to monitor the performance of model after 100 epochs. If the performance of the model degrades, then the model is stopped.

In [22]:
# Define a Callback class that stops training when the val_loss degrades above 150 epochs
# and also saves the model as checkpoints

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

In [ ]:
# Fitting the RNN to the Training set
epochs = 150
batch_size = 1
model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

In [ ]:
inputs_eval = train_sales[-timesteps:]
inputs_eval = sc.transform(inputs_eval)

inputs = train_sales[-timesteps*2:-timesteps]
inputs = sc.transform(inputs)

## Predicting the future sales for validation and evaluation periods

In [ ]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_valid.loc[1913 + j - timesteps],
                                 daysBeforeEvent2_valid.loc[1913 + j - timesteps],
                                 snap_CA_valid.loc[1913 + j - timesteps],
                                snap_TX_valid.loc[1913 + j - timesteps],
                                snap_WI_valid.loc[1913 + j - timesteps]))

    X_test = np.append(X_test, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

In [ ]:
X_eval = []
X_eval.append(inputs_eval[0:timesteps])
X_eval = np.array(X_eval)
predictions_eval = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model2.predict(X_eval[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_eval.loc[1941 + j - timesteps],
                                 daysBeforeEvent2_eval.loc[1941 + j - timesteps],
                                 snap_CA_eval.loc[1941 + j - timesteps],
                                snap_TX_eval.loc[1941 + j - timesteps],
                                snap_WI_eval.loc[1941 + j - timesteps]))

    X_eval = np.append(X_eval, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions_eval.append(predicted_stock_price)

## Post processing of future sales

For sales with predicted negative values, they are converted into zeros. 

In [ ]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))
submission = submission.T

submission_eval = pd.DataFrame(data=np.array(predictions_eval).reshape(28,30490))
submission_eval = submission_eval.T

submission = pd.concat((submission, submission_eval), ignore_index=True)

sample_submission = pd.read_csv("sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

submission[cols] = submission[cols].mask(submission[cols] < 0, 0)
result.to_csv("submission.csv", index=False)